# **Recommendation Agent powered by FloTorch:**

**Problem it solves & for whom:** It assists users who want to make smarter financial decisions by analyzing their spending and suggesting personalized savings, investment, or debt repayment strategies. It’s ideal for customers looking to improve financial health or optimize returns.

**Input**: Bank or credit card statements in PDF or CSV format, along with the areas in which you want the recommendations will be taken as a input.

**Output**: The output will be a downloadable PDF report with recommendations for each topic, based on your banking trends.

**Why it matters**: Customers lack personalized financial guidance. This agent turns raw transaction data into actionable recommendations, helping users save more, reduce debt faster, and grow their wealth intelligently.

Installs all necessary dependencies for the workflow. It includes data processing and visualization libraries (pandas, matplotlib, plotly), the flotorch framework for running agents, and tools for working with PDF files (fpdf2, pdfplumber).

In [1]:
# Install essential Python packages required for data handling, visualization, and PDF processing

# pandas: for data manipulation and analysis
# matplotlib: for creating static charts and visualizations
# plotly: for interactive data visualizations
!pip install pandas matplotlib plotly --quiet

# flotorch[strands]: for creating and running intelligent agents within the Flotorch framework
# The version 2.1.0b1 ensures compatibility with the current agent evaluation setup
!pip install flotorch[strands]==2.1.0b1

# fpdf2: used for generating downloadable PDF reports
# pdfplumber: used for extracting tables and text from PDF files
!pip install -q fpdf2 pdfplumber


INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.0
    Uninstalling wrapt-2.0.0:
      Successfully uninst

Imports the required modules for secure access and agent interaction.
It retrieves stored API credentials via google.colab.userdata and loads FlotorchStrandsAgent from the Flotorch SDK to enable communication with the Transaction Agent hosted on the Flotorch framework.

In [2]:
# Import utilities for securely handling credentials in Google Colab
# 'userdata' allows retrieval of stored API keys and tokens safely
from google.colab import userdata

# Import the FlotorchStrandsAgent class from the Flotorch framework
# This class enables interaction with a deployed AI agent through the Flotorch Gateway
from flotorch.strands.agent import FlotorchStrandsAgent


Initializes the Recommendation Agent client using the Flotorch SDK.
It connects securely to the Flotorch Gateway with the stored API key and prepares the environment for the agent to process inputs and generate financial recommendations.

In [3]:
# Initialize a FlotorchStrandsAgent client for the "Recommendation Agent"
# This object connects to the Flotorch Gateway and authenticates using the stored API key

recommendation_agent_client = FlotorchStrandsAgent(
    agent_name="recommendation-agent",           # Unique identifier for the deployed agent
    api_key=userdata.get('flotorch_api_key'),    # Securely fetches the API key stored in Colab's userdata
    base_url="https://gateway.flotorch.cloud"    # Endpoint for the Flotorch Gateway communication
)

2025-11-04 20:57:22 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 20:57:22 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 20:57:22 - flotorch.strands.llm - INFO - FlotorchStrandsModel initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 20:57:22 - flotorch.strands.llm - INFO - FlotorchStrandsModel initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


Fetches the active Recommendation Agent from the Flotorch Gateway.
Once retrieved, the agent instance can be directly invoked to analyze data and generate personalized recommendations, such as savings plans or investment suggestions.

In [4]:
# Retrieve the deployed "Recommendation Agent" instance from the Flotorch Gateway
# The returned object allows direct interaction with the agent to process prompts or data

recommendation_agent = recommendation_agent_client.get_agent()

Handles file uploads and data loading.



*  If the user uploads a CSV, it is directly read into a pandas DataFrame.  
*  If PDFs are uploaded, up to 12 files are processed with pdfplumber to extract tabular data from each page.
All extracted tables are merged into one dataset, saved as merged_from_pdfs.csv, and loaded into the variable df for further processing.

In [5]:
# === FILE UPLOAD & LOAD (PDF or CSV) ==========================================
# - If PDF: upload up to 12 PDFs, extract tables, merge, save to one CSV, and load into `df`.
# - If CSV: upload a CSV and load directly into `df`.
# Creates/overwrites:
#   - df: pandas.DataFrame -> raw loaded rows (no column normalization yet)

import io
import os
import sys
import pandas as pd

# Try to import the Colab-specific file upload helper.
# If this fails, we explicitly tell the user this cell is meant for Google Colab.
try:
    from google.colab import files
except Exception as e:
    raise RuntimeError("This cell is designed for Google Colab. `google.colab` not available.") from e

# Ask the user which type of file they are uploading.
# Only "pdf" or "csv" are accepted; anything else raises a clear error.
kind = input("Are you uploading pdf or csv? (pdf/csv): ").strip().lower()
if kind not in {"pdf", "csv"}:
    raise ValueError("Please type exactly 'pdf' or 'csv'.")

# ===================== CSV BRANCH =====================
if kind == "csv":
    print("Upload a CSV file…")
    uploaded = files.upload()  # user selects a CSV
    if not uploaded:
        raise RuntimeError("No file uploaded.")
    # Take the first uploaded file
    fname = next(iter(uploaded.keys()))
    if not fname.lower().endswith(".csv"):
        raise ValueError(f"Uploaded file '{fname}' is not a .csv")
    file_path = f"/content/{fname}"
    # Persist the uploaded CSV to disk under /content
    with open(file_path, "wb") as f:
        f.write(uploaded[fname])

    # Load into DataFrame
    df = pd.read_csv(file_path)
    print(f"Loaded CSV: {file_path} with {len(df):,} rows.")

# ===================== PDF BRANCH =====================
else:
    # kind == "pdf"
    print("Upload up to 12 PDF files…")
    uploaded = files.upload()
    if not uploaded:
        raise RuntimeError("No files uploaded.")
    # Filter only PDF files from uploaded content
    pdf_files = [name for name in uploaded.keys() if name.lower().endswith(".pdf")]
    if not pdf_files:
        raise ValueError("No PDF files detected in the upload.")
    if len(pdf_files) > 12:
        raise ValueError(f"You uploaded {len(pdf_files)} PDFs. Please upload at most 12.")

    # Write PDFs to disk so pdfplumber can read them
    pdf_paths = []
    for name in pdf_files:
        path = f"/content/{name}"
        with open(path, "wb") as f:
            f.write(uploaded[name])
        pdf_paths.append(path)

    # Install lightweight PDF table extractor (no external binaries needed)
    # We lazily install pdfplumber only if it's not already present.
    try:
        import pdfplumber  # noqa
    except ImportError:
        print("Installing pdfplumber…")
        !pip -q install pdfplumber
        import pdfplumber  # noqa

    # Extract tables from each PDF page → accumulate rows
    all_tables = []
    import pdfplumber
    for p in pdf_paths:
        print(f"Extracting tables from {os.path.basename(p)} …")
        with pdfplumber.open(p) as pdf:
            for page_idx, page in enumerate(pdf.pages):
                try:
                    # Try structured extraction using explicit line-based strategies
                    tables = page.extract_tables(table_settings={
                        "vertical_strategy": "lines",
                        "horizontal_strategy": "lines",
                        "intersection_tolerance": 5,
                    })
                except Exception:
                    # Fallback to default extraction strategy if custom settings fail
                    tables = page.extract_tables()
                for t in tables or []:
                    # Basic cleanup: skip empty or malformed tables
                    if not t or len(t) < 1:
                        continue
                    # Assume first row is header if looks header-ish; otherwise synthesize
                    header = t[0]
                    body = t[1:] if len(t) > 1 else []
                    # Heuristic: if header has many None/empty, synthesize generic headers
                    header_clean = [h if (h is not None and str(h).strip()) else f"col_{i}" for i, h in enumerate(header)]
                    df_tbl = pd.DataFrame(body, columns=header_clean)
                    # Keep table if at least two columns
                    if df_tbl.shape[1] >= 2 and df_tbl.shape[0] >= 1:
                        all_tables.append(df_tbl)

    # If we couldn’t extract any usable tables, fail fast with a helpful error.
    if not all_tables:
        raise RuntimeError("No tables detected in the uploaded PDFs. Please ensure your PDFs contain tabular data.")

    # Concatenate all extracted tables into a single DataFrame
    merged_df = pd.concat(all_tables, ignore_index=True)

    # Save to a single CSV for reproducibility
    file_path = "/content/merged_from_pdfs.csv"
    merged_df.to_csv(file_path, index=False)

    # Also expose as `df`
    df = merged_df
    print(f"Merged {len(pdf_paths)} PDFs → {file_path} with {len(df):,} rows.")


2025-11-04 20:57:22 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


2025-11-04 20:57:22 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.
Are you uploading pdf or csv? (pdf/csv): csv
Upload a CSV file…


Saving personal_transactions.csv to personal_transactions.csv
Loaded CSV: /content/personal_transactions.csv with 806 rows.


Cleans and standardizes the dataset. It ensures that every file has consistent column names for Date, Amount, and Description, even if the uploaded data used different labels. After normalizing, it previews the first 10 rows of the cleaned DataFrame to confirm successful parsing.

In [6]:
# === NORMALIZE COLUMNS & PREVIEW =============================================
# Soft-parse a date column, normalize amount column name, and ensure a description column exists.
# Expects `df` and `file_path` to already be defined by the previous upload cell.

import pandas as pd
from IPython.display import display

# --- Identify and parse date columns ---
# Look for a column that matches common date naming patterns
date_cols = [c for c in df.columns if str(c).lower() in ["date", "posted_date", "transaction_date", "datetime"]]
if date_cols:
    # Convert recognized date column to datetime format (coercing invalid entries to NaT)
    df[date_cols[0]] = pd.to_datetime(df[date_cols[0]], errors="coerce")
    # Rename first matching column to standard "Date"
    df.rename(columns={date_cols[0]: "Date"}, inplace=True)
else:
    # If no date column exists, create one filled with NaT to maintain structure
    df["Date"] = pd.NaT  # absent

# --- Normalize amount column ---
amt_col = None
# Search for a column name representing transaction amount
for c in df.columns:
    if str(c).lower() in ["amount", "amt", "value", "transaction_amount"]:
        amt_col = c
        break
# Raise an error if no recognizable amount column is found
if amt_col is None:
    raise ValueError("Could not find an amount-like column (e.g., Amount). "
                     "Please rename your amount column to 'Amount' and re-run.")

# Standardize amount column name to "Amount"
df.rename(columns={amt_col: "Amount"}, inplace=True)

# --- Ensure a description/merchant column exists ---
desc_col = None
# Try finding columns that likely contain text descriptions or merchant names
for c in df.columns:
    if str(c).lower() in ["description", "merchant", "payee", "narration", "memo"]:
        desc_col = c
        break
if desc_col is None:
    # If none exist, create a fallback "Description" column with row numbers
    df["Description"] = "Row-" + (df.reset_index().index + 1).astype(str)
    desc_col = "Description"

# --- Display summary and preview ---
print("Loaded/Prepared data from:", file_path)
print("Columns normalized to include: ['Date', 'Amount', 'Description', ...]")
display(df.head(10))  # Show first 10 rows for verification


Loaded/Prepared data from: /content/personal_transactions.csv
Columns normalized to include: ['Date', 'Amount', 'Description', ...]


,Date,Description,Amount,Transaction Type,Category,Account Name
0,2018-01-01,Amazon,11.11,debit,Shopping,Platinum Card
1,2018-01-02,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,2018-01-02,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,2018-01-03,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,2018-01-04,Netflix,11.76,debit,Movies & DVDs,Platinum Card
5,2018-01-05,American Tavern,25.85,debit,Restaurants,Silver Card
6,2018-01-06,Hardware Store,18.45,debit,Home Improvement,Silver Card
7,2018-01-08,Gas Company,45.00,debit,Utilities,Checking
8,2018-01-08,Hardware Store,15.38,debit,Home Improvement,Silver Card
9,2018-01-09,Spotify,10.69,debit,Music,Platinum Card


Prepare a compact text sample of the uploaded dataset for the language model.
It selects the first 80 rows (or fewer if the file is smaller) and converts them into CSV-formatted text, making the data concise and structured for LLM-based processing.

In [7]:
# df is already loaded from your uploaded CSV

# Limit the number of rows passed to the model to avoid excessive token usage
# (Here we cap at 80 rows to stay within safe token limits)
n_rows = min(80, len(df))  # keep tokens under control

# Extract the top n_rows from the dataframe to create a manageable sample
sample_df = df.head(n_rows)

# Convert the sample dataframe into a CSV-formatted text string
# This makes the data compact, structured, and easier for an LLM to parse
sample_text = sample_df.to_csv(index=False)

# Print confirmation message for user visibility
print("Using first", n_rows, "rows as LLM context.")

Using first 80 rows as LLM context.


Prompts the user to input financial recommendation topics such as saving strategies, debt reduction, or investment planning.
If the user does not enter any input, a set of default topics is used. The selected topics are then displayed and later sent to the Recommendation Agent to generate personalized financial advice.

In [8]:
# === RECOMMENDATION TOPIC INPUT CELL =========================================
# This cell collects user input for financial recommendation topics.
# The topics guide the Recommendation Agent in tailoring its responses.

# Display example prompts to guide the user
print("Example topics you can ask for:")
print("avoiding fees, automating savings, paying down debt efficiently, optimizing card rewards/usage")
print("You can also enter your own, like:")
print("building emergency fund, improving credit score, cutting subscription costs, investment planning\n")

# Collect user input — expects 3–6 topics, comma-separated
user_topics = input("Enter 3–6 topics you want recommendations on (comma-separated): ").strip()

# Use default examples if user leaves input blank
if not user_topics:
    user_topics = "avoiding fees, automating savings, paying down debt efficiently, optimizing card rewards/usage"

# Confirm topics selected by user
print(f"\n Topics selected for recommendations:\n{user_topics}")


Example topics you can ask for:
avoiding fees, automating savings, paying down debt efficiently, optimizing card rewards/usage
You can also enter your own, like:
building emergency fund, improving credit score, cutting subscription costs, investment planning

Enter 3–6 topics you want recommendations on (comma-separated): automating savings, paying down debt efficiently

 Topics selected for recommendations:
automating savings, paying down debt efficiently


Computes a last-6-months spending summary (if dates are available) and embeds it, along with a CSV sample of transactions and the user-selected topics, into a prompt for the Recommendation Agent. The prompt instructs the agent to generate 3–6 plain-text, bullet-style financial recommendations with expected benefit, confidence level, and a brief note on which inputs were used.

In [9]:
# Derive a simple "last 6 months spend" summary from the dataframe, if Date is available
month_series = None
if pd.notna(df["Date"]).any():
    # Drop rows with missing Date and work on a copy
    m = df.dropna(subset=["Date"]).copy()
    # Convert dates to month period strings like '2025-01'
    m["Month"] = m["Date"].dt.to_period("M").astype(str)
    # Group by Month and sum Amount, then keep only the last 6 months as a dict
    month_series = m.groupby("Month")["Amount"].sum().tail(6).to_dict()
else:
    # If there is no Date information at all, pass an empty context for month_series
    month_series = {}

# Build the natural language prompt for the Recommendation Agent
rec_prompt = f"""
You are the Recommendation Agent.

Context:
- Last 6 months spend totals (if available): {month_series}
- Sample transactions (CSV, first {len(sample_df)} rows):
---
{sample_text}
---

Suggest 3–6 actionable recommendations across:
{user_topics}

For each item, include in plain text:
- expected_benefit (qualitative),
- confidence (High/Medium/Low),
- inputs_used (brief).

Keep formatting simple (bullets). No JSON, no tables.
"""


Configures logging behavior for a cleaner notebook experience.
It removes any existing log handlers from the flotorch and strands modules and sets their logging level to WARNING, ensuring that only important messages are displayed during agent execution.

In [10]:
import logging

# Remove any existing Flotorch or Strands log handlers to prevent duplicate or cluttered logs.
for name in ["flotorch", "strands"]:
    logger = logging.getLogger(name)
    logger.handlers.clear()   # Clear any attached handlers
    logger.propagate = False  # Stop propagation to the root logger

# Reduce logging verbosity so the notebook only displays warnings and errors.
# This keeps the output clean and focused on essential information.
logging.getLogger("flotorch").setLevel(logging.WARNING)
logging.getLogger("strands").setLevel(logging.WARNING)


Invokes the Recommendation Agent with the constructed prompt containing recent spending data, transaction samples, and selected user topics.
The agent processes this context to produce personalized financial recommendations—including savings, investment, and expense optimization insights—returned in the response variable.

In [11]:
# Send the prepared prompt (rec_prompt) to the Recommendation Agent.
# The agent uses the provided context (transaction data, spending trends, and user topics)
# to generate personalized financial recommendations.

response = recommendation_agent(rec_prompt)


Based on your spending patterns and financial data, here are my recommendations:

**Automating Savings:**

• **Set up automatic transfer of $500-800 monthly to high-yield savings account**
  - Expected benefit: Build emergency fund and reduce temptation to overspend during high-spend months
  - Confidence: High
  - Inputs used: Monthly spend averaging $11,864 with $4,000 biweekly income showing capacity for savings

• **Automate 10% of each paycheck ($400) into retirement/investment account**
  - Expected benefit: Consistent wealth building through dollar-cost averaging and compound growth
  - Confidence: High
  - Inputs used: Regular $2,000 biweekly paychecks showing stable income stream

• **Create automatic "spending spike" buffer by saving extra $200 in June/high-spend months**
  - Expected benefit: Smooth out irregular expenses like your $16,514 June spending without disrupting other goals
  - Confidence: Medium
  - Inputs used: June 2019 spending 40% higher than average monthly s

This cell converts the Recommendation Agent’s output into a well-formatted PDF report.
It extracts clean text from the response, removes extra markdown or log lines, and uses fpdf2 to build a professional “Personal Finance Recommendations” document. The PDF is stored in memory and displayed as a download link for easy access within Colab.

In [12]:
from fpdf import FPDF
from fpdf.enums import XPos, YPos
from IPython.display import HTML, display
from io import BytesIO
import base64, re

# Extract raw text from the agent response
# The `response` variable contains output from: response = recommendation_agent(rec_prompt)
raw = response

# Handle different response structures (dict or object form)
if hasattr(raw, "message"):
    raw = raw.message
if isinstance(raw, dict) and "content" in raw:
    # Flotorch/Strands format → {'role': 'assistant', 'content': [{'text': "..."}]}
    try:
        raw = raw["content"][0]["text"]
    except Exception:
        raw = str(raw)

text = str(raw)

# Remove noisy sync/log lines
# These are backend synchronization messages (e.g., "[Sync] Reload interval passed...")
lines = text.splitlines()
lines = [ln for ln in lines if not ln.strip().startswith("[Sync]")]
text = "\n".join(lines)

# Clean markdown or formatting artifacts
# Removes backticks, asterisks, underscores, and hashes for a clean text output
clean_text = re.sub(r"[`*_#]+", "", text)
lines = [ln.strip() for ln in clean_text.splitlines() if ln.strip()]

# Create the PDF document
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)

# Add title section
pdf.set_font("Helvetica", style="B", size=16)
pdf.cell(0, 10, "Personal Finance Recommendations",
         new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
pdf.ln(6)

# Add introductory text
pdf.set_font("Helvetica", size=12)
pdf.multi_cell(0, 8, "Generated by Recommendation Agent based on your transaction history:")
pdf.ln(4)

# Format each line of the agent's output
for line in lines:
    # Bold style for headers, enumerations, or labeled lines
    if re.match(r"^\d+\)", line) or re.match(r"^\d+\.", line) or line.endswith(":") or "Recommendation" in line:
        pdf.set_font("Helvetica", style="B", size=12)
        pdf.multi_cell(0, 8, line)
        pdf.ln(2)
    # Bullet points (• or -) indented slightly
    elif line.startswith("•") or line.startswith("-"):
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(0, 7, "   " + line.lstrip("•- ").strip())
    # Normal lines (plain text)
    else:
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(0, 7, line)
    pdf.ln(1)

# Export PDF to memory and generate a base64 download link
buf = BytesIO()
pdf.output(buf)
pdf_bytes = buf.getvalue()

# Encode the PDF to Base64 and display a clickable download link in Colab
b64 = base64.b64encode(pdf_bytes).decode("ascii")
href = f'<a download="Recommendation_Report.pdf" href="data:application/pdf;base64,{b64}" target="_blank">Download Recommendation Report (PDF)</a>'
display(HTML(href))
